In [1]:
# !mkdir -p challenge/ 
# !wget -O challenge/distrib.zip "https://api.ctf.sg/file?id=ckm71i64d18oz0856ldrj424b&name=distrib.zip"

--2021-03-14 08:04:19--  https://api.ctf.sg/file?id=ckm71i64d18oz0856ldrj424b&name=distrib.zip
Resolving api.ctf.sg (api.ctf.sg)... 172.67.194.92, 104.21.36.126, 2606:4700:3037::6815:247e, ...
Connecting to api.ctf.sg (api.ctf.sg)|172.67.194.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63757 (62K) [application/zip]
Saving to: ‘challenge/distrib.zip’

challenge/distrib.z 100%[===================>]  62.26K  --.-KB/s    in 0.1s    

2021-03-14 08:04:20 (515 KB/s) - ‘challenge/distrib.zip’ saved [63757/63757]



In [2]:
!unzip -o challenge/distrib.zip

Archive:  challenge/distrib.zip
  inflating: distrib/eval.py         
  inflating: distrib/leenet.ph       


In [3]:
!mv distrib/* .
!rmdir distrib

In [4]:
from PIL import Image
from IPython.display import display
import numpy as np
import subprocess
import requests

import torch
import torch.nn as nn
from torch.autograd import Function, Variable
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
import torch.nn.functional as F

In [5]:
NFEATURES=16*16
NHIDDEN=8*8
NCL=5
EPS = 0.00001

# not LeNet
class LeeNet(nn.Module):
    def __init__(self, nFeatures=NFEATURES, nHidden=NHIDDEN, nCl=NCL):
        super().__init__()
        self.fc1 = nn.Linear(nFeatures, nHidden)
        self.fc2 = nn.Linear(nHidden, nCl)
        
    def forward(self, x):
        nBatch = x.size(0)

        # Normal FC network.
        x = x.view(nBatch, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Remove layer norm
        # We want x to be all 0 or negative
        # x = F.layer_norm(x, x.size()[1:], eps=1e-50)
        # probs = F.softmax(x, dim=1) 
        
        return x

In [6]:
model = LeeNet()
model.load_state_dict(torch.load("./leenet.ph"))

<All keys matched successfully>

In [7]:
for param in model.parameters():
    param.requires_grad = True

In [8]:
def step_adversarial(im_np, lr=100):
    im_tensor = torch.Tensor(im_np)
    im_tensor.requires_grad = True
    im_tensor.grad = None
    preds = model(im_tensor)
    print(preds)
    # Propagate loss on any labels that are still positive
    target = torch.empty([1, 5]).fill_(0)
    loss = nn.L1Loss()
    output = loss(preds, target)
    output.backward()
    new_im_np = (im_np - im_tensor.grad.clone().detach().numpy() * 100).clip(0, 1)
    return new_im_np

In [9]:
# Generate image!
im_np = np.random.rand(1, 256)
for i in range(10):
    im_np = step_adversarial(im_np)

tensor([[0.0000, 0.1602, 0.0770, 0.0000, 0.1955]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 0., 0., 0.]], grad_fn=<ReluBackward0>)


/opt/conda/lib/python3.8/site-packages/torch/autograd/__init__.py:145: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:104.)
  Variable._execution_engine.run_backward(


In [10]:
im_np = (im_np * 255).clip(0, 255).astype(np.uint8).reshape((16, 16))
im = Image.fromarray(im_np)
display(im)
!mkdir -p test/test
im.save('test/test/win.png')

In [11]:
print(subprocess.check_output(["python3", "eval.py", "./test/"]).decode())

0



In [12]:
res = requests.post(
    'http://which-lee.chals.ctf.sg:13337/check_lee',
    data=open('test/test/win.png', 'rb'),
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)

In [13]:
res.text

'{"class":0,"flag":"CTFSG{holee_shit_how_did_you_get_this}"}'